# Assignment 5: Deep Learning Optimizer

## Meta Instructions
1. Environment: If you use Google Colab, the default environment is OK. For your own environment, install pytorch >= 1.7.0 and torchvision >= 0.8.0 to avoid any issues caused by package versions.

2. Finish coding tasks according to instructions in this file. **Only** change the code/text in the required area, which is indicated by "Your code/answer starts here" and "Your code/answer ends here". Do not modify anywhere else including comments. 1 GPU is sufficient for this task.

2. Submission: submit a zip file named "StuID.zip" (e.g., "A0123456J.zip") to Canvas. Note that it is **NOT** NUSNET ID. The zip file should **only** include "main.ipynb" with your implemented code. **Keep the intermediate results in this notebook. We won't run it for you.** The submissison deadline is **23:59 Mar 17**.


Please **strictly** follow the above instructions, otherwise a grade deduction will be conducted.

For any question, please do one of the following actions with priority:
1. Search for similar questions on Slack
2. Propose a new question on Slack
3. For non-public questions, e-mail to Xiangyu Peng (e0792500@u.nus.edu) with the subject starting with "CS5260 2023 Spring"

## Get Started

In this assignment, we will learn to implement the [**LARS**](https://arxiv.org/abs/1708.03888) optimizer and do hyper-parameter tuning to maximally exploit the capability of the optimizer. **LARS** is short for Layer-wise Adaptive Rate Scaling. The algorithm of **LARS** is shown below.
<img src="./resources/lars.jpg" alt="Alternative text" width="900"/>

## Load the CIFAR-10 dataset

In this task, we still use CIFAR-10 for training. Let's first prepare the dataset.

In [1]:
import time
import torch
import torch.nn as nn
import torchvision
from torch.optim import lr_scheduler


class Args:
    root = './data'
    epochs = 50
    batch_size = 2048

args = Args()

In [2]:
from torchvision.datasets import CIFAR10
from torchvision import transforms

mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

train_set = CIFAR10(root=args.root, train=True, transform=transform_train, download=True)
test_set = CIFAR10(root=args.root, train=False, transform=transform_test)


train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=args.batch_size,
    num_workers=10,
    shuffle=True,
)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=args.batch_size,
    num_workers=10,
    shuffle=False,
    drop_last=False,
)
print('==> Dataloaders built.')

Files already downloaded and verified
==> Dataloaders built.


## Prepare the model

We use a small model ResNet-18 for fast training. We remove the maxpool as CIFAR-10 images are already very small.

In [3]:
model = torchvision.models.resnet18(pretrained=False, num_classes=10)
model.maxpool = nn.Identity()
model = model.cuda()

## The LARS optimizer

Next, we are going to implement the LARS optimizer.

In [4]:
import torch
from torch.optim.optimizer import Optimizer


class LARS(Optimizer):
    """
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate
        momentum (float, optional): momentum factor (default: 0)
        eta (float, optional): LARS coefficient as used in the paper (default: 1e-3)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        dampening (float, optional): dampening for momentum (default: 0)
        nesterov (bool, optional): enables Nesterov momentum (default: False)
        epsilon (float, optional): epsilon to prevent zero division (default: 0)

    Example:
        >>> optimizer = torch.optim.LARS(model.parameters(), lr=0.1, momentum=0.9)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    """

    def __init__(self, params, lr=0.1, momentum=0, eta=1e-3, dampening=0,
                 weight_decay=0, nesterov=False, epsilon=0):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))

        defaults = dict(lr=lr, momentum=momentum, eta=eta, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov, epsilon=epsilon)
        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        super(LARS, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(LARS, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    def step(self, closure=None):
        """
        Args:
            closure (callable, optional): A closure that reevaluates the model and returns the loss.
        """

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            eta = group['eta']
            dampening = group['dampening']
            nesterov = group['nesterov']
            epsilon = group['epsilon']

            for p in group['params']:
                if p.grad is None:
                    continue
                w_norm = torch.norm(p.data)
                g_norm = torch.norm(p.grad.data)
                if w_norm * g_norm > 0:
                    """
                    TODO (Task 1): compute the local learning rate according to the algorithm above.
                    Hints:
                        1. Don't forget to use the trust coefficient eta in Eq.(6), which is not shown in the main algorithm. 
                    """
                    ################################
                    # Your code starts here
                    ################################
                    local_lr = eta * w_norm / (g_norm +
                                               weight_decay * w_norm + epsilon)
                    ################################
                    # Your code ends here
                    ################################
                else:
                    local_lr = 1
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
                    else:
                        buf = param_state['momentum_buffer']
                    buf.mul_(momentum).add_(1 - dampening, d_p)
                    if nesterov:
                        d_p = d_p.add(momentum, buf)
                    else:
                        d_p = buf

                p.data.add_(-local_lr * group['lr'], d_p)

        return loss

## Training with LARS

We provide some utility functions for training.

In [5]:
def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    # millis = int(seconds*1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    # if millis > 0 and i <= 2:
    #     f += str(millis) + 'ms'
    #     i += 1
    if f == '':
        f = '0ms'
    return f


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].contiguous().view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

We have already prepared the training and testing code for you.

In [6]:
def train(train_loader, model, criterion, optimizer):
    losses = AverageMeter()
    accs = AverageMeter()

    model.train()
    for i, (input, target) in enumerate(train_loader):
        input = input.cuda()
        target = target.cuda()

        output = model(input)
        loss = criterion(output, target)

        acc = accuracy(output, target)[0]

        losses.update(loss.item(), input.size(0))
        accs.update(acc.item(), input.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    log = {
        'loss': losses.avg,
        'acc': accs.avg,
    }

    return log


def test(test_loader, model, criterion):
    losses = AverageMeter()
    accs = AverageMeter()

    model.eval()
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.cuda()
            target = target.cuda()

            output = model(input)
            loss = criterion(output, target)

            acc1, acc5 = accuracy(output, target.contiguous(), topk=(1, 5))

            losses.update(loss.item(), input.size(0))
            accs.update(acc1.item(), input.size(0))

    log = {
        'loss': losses.avg,
        'acc': accs.avg,
    }
    return log

Your task is to do some hyper-parameter tuning to achieve the best testing results. The components for tuning include the hyper-parameters of LARS and the learning rate scheduler. Do not change the number of epochs. The training time on a single GPU should be no longer than 15 minutes. You can run multiple times and keep the best result.

In [8]:
from torch.optim import lr_scheduler

"""
TODO (Task 2): do hyper-parameter tuning.
Hints:
    1: What is the most important hyper-parameter for a training recipe?
    2: Choose your own lr scheduler; The default scheduler does not change the learning rate during training.
Requirements:
    1. Use LARS as the optimizer
"""
################################
# Your code starts here
################################

# optimizer = LARS(model.parameters())
# scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[999])

optimizer = LARS(model.parameters(), 5.0, 0.95, 5e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[40], gamma=0.1)
    
################################
# Your code ends here
################################


assert isinstance(optimizer, LARS)
assert isinstance(scheduler, lr_scheduler._LRScheduler)
assert args.epochs == 50

In [9]:
criterion = nn.CrossEntropyLoss().cuda()
best_acc = 0.0

t1 = time.time()
for epoch in range(1, args.epochs + 1):

    # train one epoch
    train_log = train(train_loader, model, criterion, optimizer)
    scheduler.step()

    print(
        f'Epoch [{epoch}/{args.epochs}] - '
        f'lr: {optimizer.param_groups[0]["lr"]:.2f}, '
        f'train_loss: {train_log["loss"]:.2f}, train_acc: {train_log["acc"]:.2f}',
    )

    # test
    if epoch % 10 == 0:
        test_log = test(test_loader, model, criterion)
        print(
            f'Epoch [{epoch}/{args.epochs}] - '
            f'test_loss: {test_log["loss"]:.2f}, test_acc: {test_log["acc"]:.2f}',
        )
        best_acc = max(best_acc, test_log['acc'])
t2 = time.time()
tot = format_time(t2 - t1)
print(f'Total training time: {tot}')
print(f'Best test acc: {best_acc:.2f}')

/home/nus-pxy/anaconda3/envs/mm3d/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/torch/csrc/utils/python_arg_parser.cpp:1025.)


Epoch [1/50] - lr: 5.00, train_loss: 1.84, train_acc: 32.42
Epoch [2/50] - lr: 5.00, train_loss: 1.50, train_acc: 44.67
Epoch [3/50] - lr: 5.00, train_loss: 1.34, train_acc: 50.91
Epoch [4/50] - lr: 5.00, train_loss: 1.24, train_acc: 55.37
Epoch [5/50] - lr: 5.00, train_loss: 1.10, train_acc: 60.47
Epoch [6/50] - lr: 5.00, train_loss: 0.97, train_acc: 65.65
Epoch [7/50] - lr: 5.00, train_loss: 0.91, train_acc: 67.90
Epoch [8/50] - lr: 5.00, train_loss: 0.83, train_acc: 70.52
Epoch [9/50] - lr: 5.00, train_loss: 0.77, train_acc: 73.07
Epoch [10/50] - lr: 5.00, train_loss: 0.71, train_acc: 74.90
Epoch [10/50] - test_loss: 1.12, test_acc: 65.96
Epoch [11/50] - lr: 5.00, train_loss: 0.66, train_acc: 76.90
Epoch [12/50] - lr: 5.00, train_loss: 0.62, train_acc: 78.08
Epoch [13/50] - lr: 5.00, train_loss: 0.59, train_acc: 79.50
Epoch [14/50] - lr: 5.00, train_loss: 0.55, train_acc: 80.56
Epoch [15/50] - lr: 5.00, train_loss: 0.54, train_acc: 80.98
Epoch [16/50] - lr: 5.00, train_loss: 0.50, t

Grading rubrics for the best test acc (Task 2 only):

- [88%, 100%]: full mark + bonus 0.5
- [86%, 88%): full mark + bonus 0.3
- [80%, 86%): full mark
- [70%, 80%): 90% mark
- [50%, 70%): 80% mark
- [30%, 50%): 50% mark
- [0%, 30%): 0% mark